# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Create list of filepaths to process original event csv data files

In [2]:
# check current working directory
print(os.getcwd())

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiate empty list for rows that will be generated from each file
full_data_rows_list = []
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# get total number of rows 
print(len(full_data_rows_list))

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check number of rows in csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [2]:
# This should make a connection to a Cassandra instance your local machine 
# (Assuming 127.0.0.1, the default)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [3]:
# create keyspace
session.execute("""
    create keyspace if not exists sparkify
    with replication = {'class': 'SimpleStrategy', 'replication_factor': 1}""")

#### Set Keyspace

In [4]:
# set the keyspace for the session
session.set_keyspace('sparkify')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
# Since the first query that we need to design a table for needs to filter based on session ID and the item in the
# session, we'll use those two fields as our composite key. The arbitrary nature of the session ID, combined with
# the need to keep the item in session number affiliated with it, should provide an efficient distribution of the
# data across our potential Cassandra clusters. We'll also include he artist, song, and length in our table design
# since those are the target pieces of information that need to be returned.

session.execute("""
    CREATE TABLE IF NOT EXISTS session_library (
        sessionId INT, itemInSession INT, artist TEXT, song TEXT,
        length FLOAT, PRIMARY KEY (sessionId, itemInSession))""")


In [9]:
# Just so we can sanity-check the information that we get from, here we pull the info from our csv file
# using Pandas to verify the result we get from Cassandra later on.
df = pd.read_csv('event_datafile_new.csv')
df.loc[
    (df['sessionId'] == 338)
    & (df['itemInSession'] == 4),
    ['artist', 'song', 'length']]

,artist,song,length
444,Faithless,Music Matters (Mark Knight Dub),495.3073


In [10]:
# Set the filepath and insert the data
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO session_library (sessionId, itemInSession, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [11]:
# verify the data was entered into the table and that we get the result that we expect.

verify_query = "SELECT artist, song, length FROM session_library WHERE sessionId = 338 AND itemInSession = 4"

try:
    rows = session.execute(verify_query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)


Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [12]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

# Create table
session.execute("""
    CREATE TABLE IF NOT EXISTS user_session_library (
        userId INT, sessionId INT, itemInSession INT, artist TEXT, song TEXT, firstName TEXT, lastName TEXT,
        PRIMARY KEY ((userId, sessionId), itemInSession))""")

# Set filepath
file = 'event_datafile_new.csv'

# Insert data into table
for i, r in pd.read_csv(file).iterrows():
    values = r[['userId', 'sessionId', 'itemInSession', 'artist', 'song', 'firstName', 'lastName']].tolist()
    query = 'INSERT INTO user_session_library (userId, sessionId, itemInSession, artist, song, firstName, lastName) '
    query += 'VALUES (%s, %s, %s, %s, %s, %s, %s)'
    session.execute(query, [*values])

# Verify that query returns data
verify_query = "SELECT artist, song, firstName, lastName FROM user_session_library WHERE userId = 10 AND sessionId = 182"

try:
    rows = session.execute(verify_query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.firstname + ' ' + row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [13]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

# Create table
session.execute("""
    CREATE TABLE IF NOT EXISTS song_listeners_library (
        song TEXT, userId INT, firstName TEXT, lastName TEXT,
        PRIMARY KEY (song, userId))""")

# Read in data
data = (
    pd.read_csv('event_datafile_new.csv')
    [['song', 'userId', 'firstName', 'lastName']]
    .drop_duplicates()
    .values
    .tolist())

# Insert data into table
for r in data:
    query = 'INSERT INTO song_listeners_library (song, userId, firstName, lastName)'
    query += 'VALUES (%s, %s, %s, %s)'
    session.execute(query, [*r])

# Verify that query returns correct data
verify_query = "SELECT firstName, lastName FROM song_listeners_library WHERE song = 'All Hands Against His Own'"

try:
    rows = session.execute(verify_query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.firstname + ' ' + row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [14]:
# Drop our tables before closing out the session now that we're done
session.execute('DROP TABLE IF EXISTS session_library')
session.execute('DROP TABLE IF EXISTS user_session_library')
session.execute('DROP TABLE IF EXISTS song_listeners_library')

### Close the session and cluster connection¶

In [15]:
session.shutdown()
cluster.shutdown()